# Simulation generator V1

In [25]:
import numpy as np
import scipy
from scipy import spatial
import matplotlib.pyplot as plt
import anndata
import pandas as pd
from tqdm import tqdm,trange
from sklearn.cluster import KMeans
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


class SimulateSpatialDataGenerator:
    def __init__(
        self,
        n_obs_sqrt,n_var,
        sig_ratio=0.1,randomness=0.1,sparsity=2,cell_cluster=10,
        seed=None
        ) -> None:
        self.n_obs = n_obs_sqrt
        self.n_var = n_var
        self.randomness = randomness # controls the randomness of the data
        self.sparsity = sparsity # controls the sparse of genes; less sparse genes have higher max_vals
        self.rsig = sig_ratio # controls the ratio of significant genes
        self.n_cluster = cell_cluster
        locations = np.array([[i,j] for i in range(self.n_obs) for j in range(self.n_obs)])
        km = KMeans(
            n_clusters=cell_cluster, init='random',
            max_iter = 100,
            tol = 0.01,
        )
        km.fit(locations)
        self.cell_label = km.labels_
        # self.cell_label = np.random.choice(self.n_cluster,self.n_obs**2)
        if seed is not None:
            np.random.seed(seed)

    def intmax_scale(self,x,max_val):
        x = ((x/np.quantile(x,q=0.75))*max_val).astype(int)
        return x

    def _RWR_Sim(
        self,
        transfer_matrix,lam=5,alpha=0.2,
        n_iters=10,
    ):
        # simulate data
        data = np.random.poisson(lam, self.n_obs**2)
        for _ in range(n_iters):
            data = (1-alpha)*(data.reshape((1,-1)) @ transfer_matrix) + alpha * data.reshape((1,-1))
        return data

    def generate_single_positive(self,max_val,transfer_matrix,**kwargs):
        max_val = max(max_val,3)
        x = self._RWR_Sim(transfer_matrix,**kwargs)
        x = self.intmax_scale(x,max_val)
        return x.reshape(-1)

    def generate_single_negative(self,max_val,lam=5,**kwargs):
        x = np.random.poisson(lam, self.n_obs**2)
        x = self.intmax_scale(x,max_val)
        return x

    def generate(self, verbose=True, cluster_dependence = 0.5, **kwargs):
        locations = np.array([[i,j] for i in range(self.n_obs) for j in range(self.n_obs)])
        dist_mat = scipy.spatial.distance.cdist(locations,locations)
        max_vals = np.random.gamma(self.sparsity,5,self.n_var).astype(int)

        transfer_matrix = np.exp(
            -(
                dist_mat + \
                    self.randomness*np.random.random((self.n_obs**2,self.n_obs**2))
            )
        )
        
        # from cell label to one-hot
        cell_label = np.zeros((self.n_obs**2,self.n_cluster))
        cell_label[np.arange(self.n_obs**2),self.cell_label] = 1
        cell_type_dist_mat = scipy.spatial.distance.cdist(cell_label,cell_label,metric='dice')
        # make tansfer matrix with cell type
        # same cell type will have higher probability to transfer
        transfer_matrix = transfer_matrix * (1-cluster_dependence*cell_type_dist_mat)
        transfer_matrix /= transfer_matrix.sum(axis=1)

        pos_data = np.array([
            self.generate_single_positive(max_val,transfer_matrix,**kwargs) for max_val in tqdm(
                max_vals[:int(self.n_var*self.rsig)],
                desc = 'Generating positive data',
                disable = not verbose
                )
            ])
        
        negtive_transfer_mat = transfer_matrix.copy().flatten()
        np.random.shuffle(negtive_transfer_mat)
        negtive_transfer_mat = negtive_transfer_mat.reshape(transfer_matrix.shape)
        neg_data = np.array([self.generate_single_positive(max_val,transfer_matrix=negtive_transfer_mat,**kwargs) for max_val in max_vals[int(self.n_var*self.rsig):]])
        data = np.concatenate([pos_data,neg_data],axis=0)
        data = data.T

        adata = anndata.AnnData(
            data,
            obs = pd.DataFrame(index=[f"cell_{i}" for i in range(self.n_obs**2)]),
            var = pd.DataFrame(index=[f"gene_{i}" for i in range(self.n_var)])
            )

        adata.obs['cell_label'] = self.cell_label
        adata.obsm['spatial'] = locations
        adata.var['positive'] = np.concatenate([np.ones(int(self.n_var*self.rsig)),np.zeros(self.n_var-int(self.n_var*self.rsig))]).astype(bool)
        adata.uns['transfer_matrix'] = transfer_matrix
        simulation_params = {
            'n_obs_sqrt':self.n_obs,
            'n_var':self.n_var,
            'randomness':self.randomness,
            'sparsity':self.sparsity,
            'rsig':self.rsig,
            'lam':5,
            'alpha':0.2,
            'n_iters':10,
        }
        simulation_params.update(kwargs)
        adata.uns['simulation_params'] = simulation_params

        self.adata = adata
        return adata
        
    def save(self,file_dir):
        self.adata.write_h5ad(file_dir)

    def plot(self,value,**kwargs):
        plt.scatter(self.adata.obsm['spatial'][:,0],self.adata.obsm['spatial'][:,1],c=value,**kwargs)


class SimulateSpatialDataGeneratorv2(SimulateSpatialDataGenerator):
    def __init__(self, simulate_process = np.random.poisson, simulate_process_kwargs = dict(), **kwargs):
        super().__init__(**kwargs)
        self.data_generate_ = lambda : simulate_process(**simulate_process_kwargs, size = self.n_obs**2)
    
    def _RWR_Sim(
        self,
        transfer_matrix,alpha=0.2,
        n_iters=10,
    ):
        # simulate data
        data = self.data_generate_()
        for _ in range(n_iters):
            data = (1-alpha)*(data.reshape((1,-1)) @ transfer_matrix) + alpha * data.reshape((1,-1))
        return data

In [26]:
simulator = SimulateSpatialDataGenerator(30, 1000, cell_cluster=5)

In [28]:
adata = simulator.generate()

Generating positive data: 100%|██████████| 100/100 [00:00<00:00, 377.04it/s]


In [36]:
spmodel = Spanve.Spanve(adata)

In [37]:
spmodel.fit()

## Generate

In [ ]:
import os
save_dir = './data/Simulations/'
for n_obs_sqrt in [10,30,50,80,100,150,200,250,300]:
    for randomness in np.linspace(0,0.3,4):
        seed = int(np.random.random()*1000)
        generator = SimulateSpatialDataGenerator(
            n_obs_sqrt=n_obs_sqrt,n_var=1000,randomness=randomness,seed=seed
        )
        generator.generate()
        generator.save(
            os.path.join(save_dir,f"Sim_{n_obs_sqrt}_{randomness:.1f}_{seed}.h5ad")            
        )

# Simulation generator V2

In [1]:
!mkdir ./data/Simulation2

In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import anndata
import pandas as pd
import scanpy as sc
from scvi.distributions._negative_binomial import ZeroInflatedNegativeBinomial
import torch

from collections import Counter
import math
import random

In [2]:
### Borrow from https://github.com/ZJUFanLab/scCube/
class SPatternGenerator:

    def run(self,
            generate_meta,
            set_seed: bool = False,
            seed: int = 12345,
            celltype_key: str = 'Cell_type',
            spatial_cell_type = None,
            spatial_dim: int = 2,
            spatial_size: int = 30,
            delta: float = 25,
            lamda: float = 0.75
            ):

        """
        Generate spatial pattern of cell types with single cell resolution
        :param generate_meta: generate single cell meta from VAE
        :param set_seed: False--random True--set seed
        :param seed: random seed
        :param celltype_key: column name of celltype
        :param spatial_cell_type: select cell types with spatial pattern, if None, all cell types have spatial patterns
        :param spatial_dim: spatial pattern dim
        :param spatial_size: spatial pattern size
        :param delta: spatial pattern delta, large--big spatial pattern
        :param lamda: spatial pattern lamda (0-1), large--clear pattern small--fuzzy pattern
        :return: DataFrame of single cell meta with spatial coordinates
        """

        if set_seed:
            np.random.seed(seed)

        print('generating spatial coordinates of single cells...')

        # Generates a realisation of the Poisson point process
        sim_point = self.__create_sim_pois(spatial_dim=spatial_dim, cell_num=generate_meta.shape[0]) * spatial_size

        if int(spatial_dim) == 2:
            sim_point['grid_x'] = np.floor(np.array(sim_point['point_x'])) + 0.5
            sim_point['grid_y'] = np.floor(np.array(sim_point['point_y'])) + 0.5
            # grid
            grid = ([(x, y) for x in range(spatial_size) for y in range(spatial_size)])

        elif int(spatial_dim) == 3:
            sim_point['grid_x'] = np.floor(np.array(sim_point['point_x'])) + 0.5
            sim_point['grid_y'] = np.floor(np.array(sim_point['point_y'])) + 0.5
            sim_point['grid_z'] = np.floor(np.array(sim_point['point_z'])) + 0.5
            # grid
            grid = ([(x, y, z) for x in range(spatial_size) for y in range(spatial_size) for z in range(spatial_size)])

        # create distance matrix
        distance = pd.DataFrame(np.array(self.__cal_dist(coord=grid)))
        # Generate random variable
        mean = np.repeat(0, distance.shape[0])
        cov = np.exp(-(distance.values / delta) ** 2)
        sample = np.random.multivariate_normal(mean, cov, tol=1e-6)
        sample_dic = {}
        for i in range(len(sample)):
            sample_dic[i] = sample[i]

        grid = pd.DataFrame(grid)
        if int(spatial_dim) == 2:
            grid.columns = ['grid_x', 'grid_y']
        elif int(spatial_dim) == 3:
            grid.columns = ['grid_x', 'grid_y', 'grid_z']

        grid_out = grid + 0.5
        grid_out[celltype_key] = "unassigned"

        # get proportion of each cell type from generate_meta
        cell_type_num_dict = dict(Counter(generate_meta[celltype_key].values))
        cell_num = generate_meta.shape[0]

        for k, v in cell_type_num_dict.items():
            prop = v / cell_num
            dic1 = {key: value for key, value in sample_dic.items() if value < np.quantile(sample, prop)}
            grid_out.loc[dic1.keys(), [celltype_key]] = k
            sample_dic = {key: value for key, value in sample_dic.items() if value >= np.quantile(sample, prop)}
            sample = np.array(np.array(list(sample_dic.values())))
            cell_num = cell_num - v

        # if args.spatial_cell_type is None:
        #     print('generating spatial patterns of totally ' + str(len(cell_type_num_dict)) + ' cell types...')
        #     for k, v in cell_type_num_dict.items():
        #         prop = v / cell_num
        #         dic1 = {key: value for key, value in sample_dic.items() if value < np.quantile(sample, prop)}
        #         grid_out.loc[dic1.keys(), ['Cell_type']] = k
        #         sample_dic = {key: value for key, value in sample_dic.items() if value >= np.quantile(sample, prop)}
        #         sample = np.array(np.array(list(sample_dic.values())))
        #         cell_num = cell_num - v
        # else:
        #     choose_cell_type_list = args.spatial_cell_type
        #     print('generating spatial patterns of selected ' + str(len(choose_cell_type_list)) + ' cell types...')
        #     choose_cell_type_num_dict = dict(Counter(generate_meta.loc[generate_meta['Cell_type'].isin(choose_cell_type_list), 'Cell_type'].values))
        #     for k, v in choose_cell_type_num_dict.items():
        #         prop = v / cell_num
        #         dic1 = {key: value for key, value in sample_dic.items() if value < np.quantile(sample, prop)}
        #         grid_out.loc[dic1.keys(), ['Cell_type']] = k
        #         sample_dic = {key: value for key, value in sample_dic.items() if value >= np.quantile(sample, prop)}
        #         sample = np.array(np.array(list(sample_dic.values())))
        #         cell_num = cell_num - v

        # other cell type without spatial patterns
        # pdb.set_trace()
        # unchoose_cell_type_num_dict = dict(
        #     Counter(generate_meta.loc[~generate_meta['Cell_type'].isin(choose_cell_type_list), 'Cell_type'].values))
        # for k, v in unchoose_cell_type_num_dict.items():
        #     idx = grid_out.loc[grid_out['Cell_type'] == 'unassigned', ].index
        #     grid_num = int(np.round(v / generate_meta.shape[0] * grid_out.shape[0]))
        #     if len(idx) >= grid_num:
        #         idx_choose = np.random.choice(idx, grid_num, replace=False).tolist()
        #     else:
        #         idx_choose = np.random.choice(idx, grid_num, replace=True).tolist()
        #     grid_out.loc[idx_choose, 'Cell_type'] = k

        # handle unassigned
        cell_type_name = np.unique(generate_meta[celltype_key].values)
        replace_num = len(grid_out.loc[grid_out[celltype_key] == 'unassigned', [celltype_key]])
        grid_out.loc[grid_out[celltype_key] == 'unassigned', [celltype_key]] = np.random.choice(cell_type_name,
                                                                                                replace_num)

        if spatial_cell_type is None:
            print('generating spatial patterns of totally ' + str(len(cell_type_num_dict)) + ' cell types...')
            spa_pattern = pd.merge(grid_out, sim_point, how='inner')
        else:
            choose_cell_type_list = spatial_cell_type
            print('generating spatial patterns of selected ' + str(len(choose_cell_type_list)) + ' cell types...')
            spa_pattern = pd.merge(grid_out, sim_point, how='inner')
            # random shuffle other cell type coordinates
            idx = spa_pattern.loc[~spa_pattern[celltype_key].isin(choose_cell_type_list), ].index
            idx_cell_type = spa_pattern.loc[
                ~spa_pattern[celltype_key].isin(choose_cell_type_list), celltype_key].values.tolist()
            random.shuffle(idx_cell_type)
            spa_pattern.loc[idx, [celltype_key]] = idx_cell_type

        # correct cell type prop
        generate_cell_type_num_dict = dict(Counter(spa_pattern[celltype_key].values))
        add_dict = {}
        sub_dict = {}
        for i in list(cell_type_name):
            if generate_cell_type_num_dict[i] < cell_type_num_dict[i]:
                add_dict[i] = cell_type_num_dict[i] - generate_cell_type_num_dict[i]
            elif generate_cell_type_num_dict[i] > cell_type_num_dict[i]:
                sub_dict[i] = generate_cell_type_num_dict[i] - cell_type_num_dict[i]

        # pdb.set_trace()

        correct_index = []
        for k, v in sub_dict.items():
            correct_index.extend(
                np.random.choice(spa_pattern[spa_pattern[celltype_key] == k].index.values, v, replace=False).tolist())

        add_ct_list = []
        for k, v in add_dict.items():
            add_ct_list.extend([k] * v)

        random.shuffle(add_ct_list)
        spa_pattern.loc[correct_index, [celltype_key]] = add_ct_list

        # pdb.set_trace()

        # lamda (0-1): large--clear pattern small--fuzzy pattern
        lamda_len = round(len(spa_pattern) * (1 - lamda))
        select_idx = np.random.choice(spa_pattern.index.values, lamda_len, replace=False)
        change_idx = np.random.choice(select_idx, len(select_idx), replace=False)
        spa_pattern.loc[select_idx, [celltype_key]] = np.array(spa_pattern.loc[change_idx, celltype_key])

        # match Cell - Cell_type - coord
        generate_meta.sort_values(by=[celltype_key], inplace=True)
        spa_pattern.sort_values(by=[celltype_key], inplace=True)
        generate_meta.set_index(generate_meta['Cell'], inplace=True)
        spa_pattern.set_index(generate_meta['Cell'], inplace=True)
        if int(spatial_dim) == 2:
            generate_meta = pd.concat([generate_meta, spa_pattern.iloc[:, 3:]], axis=1)
        elif int(spatial_dim) == 3:
            generate_meta = pd.concat([generate_meta, spa_pattern.iloc[:, 4:]], axis=1)

        return generate_meta

    def __cal_dist(self, coord):
        """
        Calculate Euclidean distance between points
        """
        dis = []
        for i in range(len(coord)):
            dis.append([])
            for j in range(len(coord)):
                dis[i].append(math.dist(coord[i], coord[j]))
        return dis

    def __create_sim_pois(self,
                        spatial_dim: int,
                        cell_num: int):
        """
        Generates a realisation of the Poisson point process
        :param spatial_dim: spatial pattern dimensionality, 2 or 3
        :param cell_num: the number of points
        :return: DataFrame of spatial coordinates of points
        """
        assert spatial_dim in [2, 3], "error define spatial pattern dim!!"

        # if args.set_seed:
        #     np.random.seed(args.random_seed)

        if int(spatial_dim) == 2:
            xx = np.random.uniform(0, 1, cell_num)
            yy = np.random.uniform(0, 1, cell_num)
            out = pd.DataFrame({'point_x': xx, 'point_y': yy})
        elif int(spatial_dim) == 3:
            xx = np.random.uniform(0, 1, cell_num)
            yy = np.random.uniform(0, 1, cell_num)
            zz = np.random.uniform(0, 1, cell_num)
            out = pd.DataFrame({'point_x': xx, 'point_y': yy, 'point_z': zz})
        return out

In [3]:
class SimulateSpatialDataGenerator:
    def __init__(
        self,
        n_obs,
        n_var,
        svg_ratio=0.1,
        rand_noise=0.1,
        drop_out=0.01,
        n_cluster=5,
        seed=None
    ) -> None:
        self.n_obs = n_obs
        self.n_var = n_var
        self.rand_noise = rand_noise
        self.drop_out = drop_out # controls the sparse of genes; less sparse genes have higher max_vals
        self.n_svg = int(self.n_var * svg_ratio) # controls the ratio of significant genes
        self.n_cluster = n_cluster

        # assign each svg to a cluster
        var = pd.DataFrame(index=[f"gene_{i}" for i in range(self.n_var)])
        svg_label = np.random.randint(0,self.n_cluster,self.n_svg)
        var['gene_label'] = np.concatenate([svg_label, -np.ones(self.n_var-self.n_svg)]).astype(int).astype(str)

        self.var = var

        self.seed = seed
        if seed is not None:
            np.random.seed(seed)

    def generate_obs(
            self,
            spatial_size=30,
            delta=25,
            lamda=0.75
        ):
        cell_id = np.array([f"cell_{i}" for i in range(self.n_obs)])
        cell_label = np.random.randint(0,self.n_cluster,self.n_obs)

        obs = pd.DataFrame(
            index = cell_id,
            data = {
                'cell_type':cell_label.astype(str),
                'Cell': cell_id,
            }
        )

        obs = SPatternGenerator().run(
            generate_meta=obs,
            set_seed=self.seed is not None,
            seed=self.seed,
            celltype_key='cell_type',
            spatial_cell_type=None,
            spatial_dim=2,
            spatial_size=spatial_size,
            delta=delta,
            lamda=lamda
        )

        return obs.sort_index()

    def generate_dist(self, expr_shape=5):
        """
        Generate zero-inflated negative binomial distribution.
        including: mu (mean val), theta (inverse-divergence), dropout_rate (zero-inflated rate), scale (not used)
        """
        if not hasattr(self,'obs'):
            raise ValueError('Please generate obs first! Call generate_obs()')

        # mu: 
        #   for svgs relate to genes and cell-type
        #   for non-svgs relate to genes
        mu_simple = np.random.gamma(expr_shape, 2, self.n_var)
        # mu_svg = np.random.gamma(10, 1, self.n_svg)
        svg_foldchange = np.power(2, np.random.normal(1.5,0.25,self.n_svg))

        # mu: n_obs x n_var
        mu = np.repeat(mu_simple.reshape(1,-1),self.n_obs,axis=0)
        for gi in range(self.n_svg):
            cl = self.var['gene_label'][gi]
            cl_cells = np.where(self.obs['cell_type'] == cl)[0]
            mu[cl_cells,gi] = mu_simple[gi] * svg_foldchange[gi]

        theta = np.random.normal(20,1,self.n_var)
        # from prob to logits (drop_out)
        dropout_logits = np.log(self.drop_out/(1-self.drop_out)) * np.ones(self.n_var)

        dist = ZeroInflatedNegativeBinomial(
            mu=torch.from_numpy(mu),
            theta=torch.from_numpy(theta),
            zi_logits=torch.from_numpy(dropout_logits),
        )
        # dist = NegativeBinomial(
        #     mu=torch.from_numpy(mu),
        #     theta=torch.from_numpy(theta),
        # )
        return dist
    
    def generate_transfer_mat(self, randomness=0.1):
        locations = self.obs[['point_x','point_y']].values
        dist_mat = scipy.spatial.distance.cdist(locations,locations)
        # cell label matrix. one-hot
        labels = self.obs['cell_type'].values
        cell_mat = (labels[:, None] == labels[None, :]).astype(int)

        transfer_matrix = np.exp(
            -(
                dist_mat + \
                randomness*np.random.random((self.n_obs,self.n_obs))
            )
        )
        transfer_matrix = transfer_matrix * cell_mat
        transfer_matrix /= transfer_matrix.sum(axis=1)
        return transfer_matrix

    def generate_rwr(self, transfer_mat, mu, n_iters=10, alpha=0.5):

        # mu: n_var x n_obs 
        # transfer_mat: n_obs x n_obs
        premu = mu.copy()
        mu = mu[0:self.n_svg, :] # only update svgs
        for i in range(n_iters):
            mu = (1-alpha) * (mu @ transfer_mat) + alpha * mu
        for si in range(self.n_svg):
            cl = self.var['gene_label'][si]
            cl_cells = np.where(self.obs['cell_type'] == cl)[0]
            premu[si, cl_cells] = mu[si, cl_cells]

        return premu

    def generate(
        self, 
        spatial_size=30,
        delta=10,
        lamda=1,
        spatial_randomness=0.1,
        n_iters=10, alpha=0.5
    ):
        if not hasattr(self,'obs'):
            self.obs = self.generate_obs(
                spatial_size=spatial_size,
                delta=delta,
                lamda=lamda
            )
        if not hasattr(self,'dist'):
            self.dist = self.generate_dist()
        transfer_mat = self.generate_transfer_mat(randomness=spatial_randomness)
        mu = self.dist.mu.numpy() # n_obs x n_var
        newmu = self.generate_rwr(transfer_mat, mu.T, n_iters=n_iters, alpha=alpha).T
        self.dist.mu = torch.from_numpy(newmu)
        data = self.dist.sample().numpy()
        def intmax_scale(x, max_val):
            x = ((x/np.quantile(x,q=0.8))*max_val).astype(int)
            return x
        expr_max = np.random.gamma(2, 5, self.n_var)
        # for gene in svgs, make expr_max larger (at least 5)
        expr_max[0:self.n_svg] = np.maximum(expr_max[0:self.n_svg], 5)
        
        data = intmax_scale(data, expr_max)
        random_noise = np.random.negative_binomial(n = 1, p=1-self.rand_noise, size=(self.n_obs,self.n_var))
        data = data + random_noise 
    
        adata = anndata.AnnData(
            data,
            obs = self.obs,
            var = self.var,
            obsm={'spatial':self.obs[['point_x','point_y']].values},
            uns=dict(
                simulation_params = {
                    'spatial_randomness':spatial_randomness,
                    'spatial_size':spatial_size,
                    'delta':delta,
                    'lamda':lamda,
                    'seed':self.seed,
                    'rand_noise':self.rand_noise,
                    'drop_out':self.drop_out,
                    'n_cluster':self.n_cluster,
                },
            )
        )
        self.adata = adata
        return adata
        
    def save(self,file_dir):
        self.adata.write_h5ad(file_dir)

    def plot(self,value,**kwargs):
        plt.scatter(self.adata.obsm['spatial'][:,0],self.adata.obsm['spatial'][:,1],c=value,**kwargs)

    def plot_pattern(self):
        return self.plot('cell_type')


## generate

In [4]:
# params
default_config = dict(
    ## general
    n_obs = 5000,
    n_var = 100,
    svg_ratio = 0.5,
    n_cluster = 5,
    seed = None,
    ## noise
    rand_noise = 0, # 0, 0.25, 0.5
    drop_out = 0, # 0, 0.15, 0.3
    ## spatial affects
    spatial_randomness = 3,
    n_iters=10,
    alpha=0.5,
    spatial_size=30,
    delta=25,
)

pat_config = dict(spatial_size=30, delta=25, spatial_randomness = 3)

for rn in [0, 0.1, 0.2, 0.3]:
    for dp in [0, 0.1, 0.2, 0.3]:
        print(rn, dp)
        conf = default_config.copy()
        conf.update(pat_config)
        conf['rand_noise'] = rn
        conf['drop_out'] = dp
        simulator = SimulateSpatialDataGenerator(
            n_obs=5000,
            n_var=100,
            svg_ratio=0.5,
            rand_noise=rn,
            drop_out=dp,
            n_cluster=5,
            seed=np.random.randint(0, 10000),
        )
        adata = simulator.generate(**pat_config)
        adata.uns['simulation_params'] = conf
        adata.write_h5ad(f'./data/Simulation2/sim_{rn:.1f}_{dp:.1f}.h5ad')

0 0
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...


/tmp/ipykernel_26352/4180614710.py:85: RuntimeWarning: divide by zero encountered in log
  dropout_logits = np.log(self.drop_out/(1-self.drop_out)) * np.ones(self.n_var)


0 0.1
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...
0 0.2
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...
0 0.3
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...
0.1 0
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...


/tmp/ipykernel_26352/4180614710.py:85: RuntimeWarning: divide by zero encountered in log
  dropout_logits = np.log(self.drop_out/(1-self.drop_out)) * np.ones(self.n_var)


0.1 0.1
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...
0.1 0.2
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...
0.1 0.3
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...
0.2 0
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...


/tmp/ipykernel_26352/4180614710.py:85: RuntimeWarning: divide by zero encountered in log
  dropout_logits = np.log(self.drop_out/(1-self.drop_out)) * np.ones(self.n_var)


0.2 0.1
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...
0.2 0.2
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...
0.2 0.3
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...
0.3 0
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...


/tmp/ipykernel_26352/4180614710.py:85: RuntimeWarning: divide by zero encountered in log
  dropout_logits = np.log(self.drop_out/(1-self.drop_out)) * np.ones(self.n_var)


0.3 0.1
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...
0.3 0.2
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...
0.3 0.3
generating spatial coordinates of single cells...
generating spatial patterns of totally 5 cell types...
